In [ ]:
from eagle_hill_fund.server.tools.financial.data.tool import FinancialDataCompiler
from eagle_hill_fund.server.tools.financial.strategies.mean_reversion.tool import BollingerBandMeanReversionTool
from eagle_hill_fund.server.tools.financial.strategies.tool import BaseStrategyTool

In [ ]:
import pandas as pd

In [ ]:
ehf_data_tool = FinancialDataCompiler()

In [ ]:
exchanges = ehf_data_tool.get_available_exchanges()

In [ ]:
stock_list = ehf_data_tool.screen_companies(exchange="NYSE")

In [ ]:
stock_list = [stock["symbol"] for stock in stock_list]

In [ ]:
partial_stock_list = pd.Series(stock_list).sample(1000).tolist()

In [ ]:
price_data = ehf_data_tool.compile_price_data(
    symbols=partial_stock_list,
    from_date="1995-09-01",
    to_date="2025-09-15",
    interval="1day"
)

In [ ]:
# Import the configuration class
from eagle_hill_fund.server.tools.financial.strategies.trend.tool import DipBuyingConfig, DipBuyingTool

from datetime import datetime
# Create configuration
config = DipBuyingConfig(
    name="MyDipStrategy",
    symbols=partial_stock_list,
    dip_threshold=0.05,      # Buy on 3% dips (more frequent)
    profit_target=0.0005,      # Sell at 5% profit (more frequent exits)
    stop_loss=0.25,          # Stop loss at 10% (less restrictive)
    lookback_period=10,      # 10-day lookback (more signals)
    min_volume_ratio=0.5,    # Allow lower volume (less restrictive)
    min_price=1.0,           # Allow cheaper stocks
    max_price=10000.0,       # Allow expensive stocks
    max_holding_days=252,     # Hold up to 60 days (less restrictive)
    position_size_pct=1/len(partial_stock_list),   # 20% of portfolio per position (larger positions)
    max_positions=len(partial_stock_list)         # Allow up to 20 open positions
)

# Create and run strategy
strategy = DipBuyingTool(config)

In [ ]:
# Load data into the strategy
strategy.set_data(price_data)

# Check data loading
print(f"Data loaded: {strategy.data is not None}")
print(f"Number of symbols: {len(strategy.config.symbols)}")
print(f"Date range: {strategy.data['date'].min()} to {strategy.data['date'].max()}")
print(f"Total data points: {len(strategy.data)}")

# Show sample data
print("\nSample data:")
print(strategy.data.head())


In [ ]:
# Run a full backtest
print("Running backtest...")
print("This may take a few minutes...")

# Run the backtest
strategy.run_backtest()

In [ ]:
# Get results
results = strategy.get_performance_summary()

print("\n=== BACKTEST RESULTS ===")
print(f"Strategy: {results['strategy_name']}")
print(f"Total Return: {results['total_return']:.2%}")
print(f"Annualized Return: {results['annualized_return']:.2%}")
print(f"Sharpe Ratio: {results['sharpe_ratio']:.3f}")
print(f"Max Drawdown: {results['max_drawdown']:.2%}")
print(f"Win Rate: {results['win_rate']:.2%}")
print(f"Total Trades: {results['total_trades']}")
print(f"Final Portfolio Value: ${results['final_value']:,.2f}")

# Show some trade details
print(f"\nFirst 10 trades:")
execution_log = strategy.execution_log[:10]
for trade in execution_log:
    print(f"  {trade['timestamp']}: {trade['symbol']} {trade['action']} {trade['quantity']} @ ${trade['price']:.2f}")

In [ ]:
results

In [ ]:
# SIMPLE FIX: Just run this cell instead of the broken one above
import plotly.graph_objs as go
from plotly.subplots import make_subplots
import numpy as np

# Get portfolio snapshots for plotting
snapshots = strategy.portfolio.snapshots
dates = [snapshot.timestamp for snapshot in snapshots]
portfolio_values = [snapshot.total_value for snapshot in snapshots]

# Create a simple 2x2 plot without the problematic 4th subplot
fig = make_subplots(
    rows=2, cols=2,
    subplot_titles=(
        "Portfolio Value Over Time",
        "Drawdown Over Time",
        "Distribution of Monthly Returns",
        "Strategy Info"
    )
)

# Portfolio value
fig.add_trace(go.Scatter(x=dates, y=portfolio_values, mode='lines', name='Portfolio Value'), row=1, col=1)

# Drawdown
drawdowns = []
peak = portfolio_values[0]
for value in portfolio_values:
    if value > peak:
        peak = value
    drawdown = (value - peak) / peak
    drawdowns.append(drawdown)

fig.add_trace(go.Scatter(x=dates, y=drawdowns, fill='tozeroy', mode='lines', name='Drawdown'), row=1, col=2)

# Monthly returns
monthly_returns = []
for i in range(1, len(portfolio_values)):
    monthly_return = (portfolio_values[i] - portfolio_values[i-1]) / portfolio_values[i-1]
    monthly_returns.append(monthly_return)

fig.add_trace(go.Histogram(x=monthly_returns, nbinsx=30, name='Returns'), row=2, col=1)

# Strategy info as text in 4th subplot
strategy_info = strategy.get_strategy_info()

# Add text as a scatter plot in the 4th subplot
fig.add_trace(go.Scatter(
    x=[0.5], y=[0.5], 
    text=[""],
    mode='text',
    textfont=dict(size=12, family="monospace"),
    showlegend=False
), row=2, col=2)

# Update layout
fig.update_layout(
    height=800, width=1200,
    title_text="Strategy Performance Overview",
    showlegend=False,
    template="plotly_white"
)

# Hide axes for info panel
fig.update_xaxes(visible=False, row=2, col=2)
fig.update_yaxes(visible=False, row=2, col=2)

fig.show()
